In [ ]:
file_path = "C:\\Users\\ekino\\OneDrive - UW\\GPR\\Data\\seattle_2000_all_obs.csv"

In [ ]:
%matplotlib inline
#import skmob
import pandas as pd
#import skmob.measures.individual as ind_measure
import torch
import gpytorch
from gpytorch.kernels import RQKernel as RQ, RBFKernel as SE, \
PeriodicKernel as PER, ScaleKernel, LinearKernel as LIN, MaternKernel as MAT, \
SpectralMixtureKernel as SMK, PiecewisePolynomialKernel as PPK, CylindricalKernel as CYL
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from gpytorch.constraints import Interval
import time

# Import intra-package scripts
import utils.helper_func as helper_func
import utils.GP as GP
import mobileDataToolkit.analysis as analysis
import mobileDataToolkit.preprocessing_v2 as preprocessing

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
# Check count of observations for each unique user (use groupby)
df_grouped = df.groupby('UID').count()['unix_min']

In [ ]:
# Filter out users that do not meet certain criteria
# Users with more than 10000 observations
df_grouped = df_grouped[df_grouped > 10000]

df = df[df['UID'].isin(df_grouped.index)]

In [ ]:
# Users with at least 3 months of data
df_timeWindow = df.groupby('UID').agg({'unix_min': ['min', 'max']})
df_timeWindow = df_timeWindow[df_timeWindow['unix_min']['max'] - df_timeWindow['unix_min']['min'] > 60*24*30*3]

df = df[df['UID'].isin(df_timeWindow.index)]

In [ ]:
import random
# Get unique user IDs
user_ids = df['UID'].unique()

# Sample 50 users
sampled_user_ids = random.sample(list(user_ids), 50)

# Filter out users that are not in the sampled list
df = df[df['UID'].isin(sampled_user_ids)]

In [ ]:
df.columns

In [ ]:
# Define the new order of the columns
new_order = ['UID', 'orig_lat', 'orig_long', 'orig_unc', 'datetime', 'unix_min']

# Reorder the columns using the reindex method
df = df.reindex(columns=new_order)

# Save the dataframe to a new csv file
df.to_csv('C:\\Users\\ekino\\OneDrive - UW\\GPR\\Data\\seattle_2000_all_obs_sampled.csv', index=False)